In [8]:
import sys
sys.path.append('/home/v-runmao/projects/DomShift-ATMF')
from importlib import reload
import helper
reload(helper)
from helper import combine_all_csv
import pandas as pd


pd.set_option('display.max_rows', None)
arg_names = ['algorithm', 'target', 'lr', 'wd', 'lr2', 'wd2', 'alpha_d',
             'alpha_meta', 'hidden_size_d', 'num_hidden_d', 'n_iter_d',
             'penalty_weight', 'penalty_anneal_iters']


"""Find best hparams and epoch
"""
def model_selection(table, metric='avg_acc', split='valid'):
    search_over = ['hparams', 'Epoch']
    if metric == 'avg_loss':
        apply = lambda df: df.reset_index(level=search_over).iloc[df[(split, metric)].argmin()]
    else:
        apply = lambda df: df.reset_index(level=search_over).iloc[df[(split, metric)].argmax()]
    best = table.groupby(level=['tgt', 'algo']).apply(apply)
    best = best.set_index(['hparams', 'Epoch'], append=True)
    
    # compute average over targets
    avg = best.groupby('algo').mean()
    avg.index = pd.MultiIndex.from_tuples([('average', best.index[0][1], '', '')], names=best.index.names)
    
    return best.append(avg)


"""Compare results of specified algorithms
"""
def compare_algo(roots, metric, arg_names, oracle=False):
    comp = []
    for root in roots:
        algo_res = combine_all_csv(root, arg_names)
        algo_bst = model_selection(algo_res, metric=metric, split='test' if oracle else 'valid')
        comp.append(algo_bst)
    return pd.concat(comp).sort_index()

# Compare among algorithms

In [9]:
comparison = compare_algo(['../pt/ERM_search/',
                           '../pt/DANN_search/',
                           '../pt/MLDG_search/',
                           '../pt/IRM_search'],
                          'avg_acc',
                          arg_names)
display(comparison)

train  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 1      0.152244   
        ERM  lr=2e-05, wd=0.01                                  3      0.016935   
        IRM  penalty_weight=100.0, penalty_anneal_iters=1000... 0      0.196057   
        MLDG lr2=3e-05, wd2=0.1, alpha_meta=2.0, lr=2e-05, w... 2      0.015535   
F       DANN alpha_d=0.5, hidden_size_d=512, num_hidden_d=2,... 3      0.025878   
        ERM  lr=2e-05, wd=0.1                                   3      0.014027   
        IRM  penalty_weight=100.0, penalty_anneal_iters=1000... 0      0.249654   
        MLDG lr2=3e-05, wd2=0.1, alpha_meta=0.1, lr=2e-05, w... 2      0.034732   
M       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=2... 3      0.024485   
        ERM  lr=2e-05, wd=0.01                                  3      0.011180   
        IRM  penalty_weight=100.0, penalty_anneal_iters=1000... 0      0.193463   
        MLDG lr2=2e-05, wd2=0.01, alpha_meta=2.0, lr=2e-05, ... 2      0.024976   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.043581   
        ERM  lr=2e-05, wd=0.1                                   3      0.022436   
        IRM  penalty_weight=100.0, penalty_anneal_iters=1000... 0      0.225917   
        MLDG lr2=2e-05, wd2=0.1, alpha_meta=0.1, lr=2e-05, w... 2      0.028945   
average DANN                                                           0.061547   
        ERM                                                            0.016145   
        IRM                                                            0.216273   
        MLDG                                                           0.026047   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 1      0.951567   
        ERM  lr=2e-05, wd=0.01                                  3      0.996378   
        IRM  penalty_weight=100.0, penalty_anneal_iters=1000... 0      0.925241   
        MLDG lr2=3e-05, wd2=0.1, alpha_meta=2.0, lr=2e-05, w... 2      0.997062   
F       DANN alpha_d=0.5, hidden_size_d=512, num_hidden_d=2,... 3      0.994130   
        ERM  lr=2e-05, wd=0.1                                   3      0.996945   
        IRM  penalty_weight=100.0, penalty_anneal_iters=1000... 0      0.898699   
        MLDG lr2=3e-05, wd2=0.1, alpha_meta=0.1, lr=2e-05, w... 2      0.993112   
M       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=2... 3      0.996106   
        ERM  lr=2e-05, wd=0.01                                  3      0.997718   
        IRM  penalty_weight=100.0, penalty_anneal_iters=1000... 0      0.922254   
        MLDG lr2=2e-05, wd2=0.01, alpha_meta=2.0, lr=2e-05, ... 2      0.994653   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.988276   
        ERM  lr=2e-05, wd=0.1                                   3      0.995416   
        IRM  penalty_weight=100.0, penalty_anneal_iters=1000... 0      0.908594   
        MLDG lr2=2e-05, wd2=0.1, alpha_meta=0.1, lr=2e-05, w... 2      0.994427   
average DANN                                                           0.982520   
        ERM                                                            0.996614   
        IRM                                                            0.913697   
        MLDG                                                           0.994813   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A    

# Reproduce previous results

In [4]:
best_full = combine_all_csv('../pt/BEST', None)

In [5]:
best = best_full.groupby(level=['tgt', 'algo', 'hparams']).apply(lambda df: df.reset_index(level='Epoch').iloc[
    df[('valid', 'avg_acc')].argmax()
])
best = best.set_index('Epoch', append=True)

avg = best.groupby(level='algo').mean()
avg.index = pd.MultiIndex.from_tuples(zip(['avg']*3, avg.index, ['']*3, ['']*3), names=best.index.names)
best = best.append(avg)

display(best)

train  \
                                                                   avg_loss   
tgt algo hparams                                            Epoch             
A   DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 1.0    0.152244   
    ERM  lr=2e-05, wd=0.01                                  3.0    0.016935   
    MLDG lr2=3e-05, wd2=0.1, alpha_meta=2.0, lr=2e-05, w... 2.0    0.015535   
F   DANN alpha_d=0.5, hidden_size_d=512, num_hidden_d=2,... 3.0    0.025878   
    ERM  lr=2e-05, wd=0.1                                   3.0    0.014027   
    MLDG lr2=3e-05, wd2=0.1, alpha_meta=0.1, lr=2e-05, w... 2.0    0.034732   
M   DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=2... 3.0    0.024485   
    ERM  lr=2e-05, wd=0.01                                  3.0    0.011180   
    MLDG lr2=2e-05, wd2=0.01, alpha_meta=2.0, lr=2e-05, ... 2.0    0.024976   
T   DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2.0    0.043581   
    ERM  lr=2e-05, wd=0.1                                   3.0    0.022436   
    MLDG lr2=2e-05, wd2=0.1, alpha_meta=0.1, lr=2e-05, w... 2.0    0.028945   
avg DANN                                                           0.061547   
    ERM                                                            0.016145   
    MLDG                                                           0.026047   

                                                                             \
                                                                    avg_acc   
tgt algo hparams                                            Epoch             
A   DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 1.0    0.951567   
    ERM  lr=2e-05, wd=0.01                                  3.0    0.996378   
    MLDG lr2=3e-05, wd2=0.1, alpha_meta=2.0, lr=2e-05, w... 2.0    0.997062   
F   DANN alpha_d=0.5, hidden_size_d=512, num_hidden_d=2,... 3.0    0.994130   
    ERM  lr=2e-05, wd=0.1                                   3.0    0.996945   
    MLDG lr2=3e-05, wd2=0.1, alpha_meta=0.1, lr=2e-05, w... 2.0    0.993112   
M   DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=2... 3.0    0.996106   
    ERM  lr=2e-05, wd=0.01                                  3.0    0.997718   
    MLDG lr2=2e-05, wd2=0.01, alpha_meta=2.0, lr=2e-05, ... 2.0    0.994653   
T   DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2.0    0.988276   
    ERM  lr=2e-05, wd=0.1                                   3.0    0.995416   
    MLDG lr2=2e-05, wd2=0.1, alpha_meta=0.1, lr=2e-05, w... 2.0    0.994427   
avg DANN                                                           0.982520   
    ERM                                                            0.996614   
    MLDG                                                           0.994813   

                                                                             \
                                                                         f1   
tgt algo hparams                                            Epoch             
A   DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 1.0    0.941866   
    ERM  lr=2e-05, wd=0.01                                  3.0    0.995687   
    MLDG lr2=3e-05, wd2=0.1, alpha_meta=2.0, lr=2e-05, w... 2.0    0.997296   
F   DANN alpha_d=0.5, hidden_size_d=512, num_hidden_d=2,... 3.0    0.993099   
    ERM  lr=2e-05, wd=0.1                                   3.0    0.996414   
    MLDG lr2=3e-05, wd2=0.1, alpha_meta=0.1, lr=2e-05, w... 2.0    0.992812   
M   DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=2... 3.0    0.994923   
    ERM  lr=2e-05, wd=0.01                                  3.0    0.997025   
    MLDG lr2=2e-05, wd2=0.01, alpha_meta=2.0, lr=2e-05, ... 2.0    0.994268   
T   DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2.0    0.990159   
    ERM  lr=2e-05, wd=0.1                                   3.0    0.996155   
    MLDG lr2=2e-05, wd2=0.1, alpha_meta=0.1, lr=2e-05, w... 2.0    0.995640   
avg DANN                               

# Analyze search of DANN variants

### DANN with K

In [16]:
dann = combine_all_csv('../pt/DANN_search_K/', arg_names)
model_selection(dann, metric='avg_acc')

train  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.040794   
F       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 1      0.116311   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.013674   
T       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 3      0.026611   
average DANN                                                           0.049347   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.988376   
F       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 1      0.961488   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.997248   
T       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 3      0.994887   
average DANN                                                           0.985500   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.986172   
F       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 1      0.954606   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.996414   
T       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 3      0.995714   
average DANN                                                           0.983227   

                                                                          valid  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.204537   
F       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 1      0.207619   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.223517   
T       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 3      0.293412   
average DANN                                                           0.232271   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.941335   
F       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 1      0.934340   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.950309   
T       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 3      0.930183   
average DANN                                                           0.939042   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.933180   
F       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 1      0.924393   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.936097   
T       DANN alpha_d=0.5, hidden_size_d=1024, num_hidden_d=1... 3      0.942308   
average DANN                                                           0.933994   

                                                                           test  \
 

### uniformly regularized DANN

In [18]:
dann = combine_all_csv('../pt/DANN_uniform_search/', arg_names)
model_selection(dann, metric='avg_acc')

train  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=1.0, hidden_size_d=512, num_hidden_d=2,... 3      0.137189   
F       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=2,... 3      0.034214   
M       DANN alpha_d=0.75, hidden_size_d=512, num_hidden_d=2... 3      0.031537   
T       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=1,... 3      0.039765   
average DANN                                                           0.060677   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=1.0, hidden_size_d=512, num_hidden_d=2,... 3      0.990061   
F       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=2,... 3      0.993831   
M       DANN alpha_d=0.75, hidden_size_d=512, num_hidden_d=2... 3      0.996173   
T       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=1,... 3      0.991978   
average DANN                                                           0.993011   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=1.0, hidden_size_d=512, num_hidden_d=2,... 3      0.988134   
F       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=2,... 3      0.992745   
M       DANN alpha_d=0.75, hidden_size_d=512, num_hidden_d=2... 3      0.995014   
T       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=1,... 3      0.993272   
average DANN                                                           0.992291   

                                                                          valid  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=1.0, hidden_size_d=512, num_hidden_d=2,... 3      0.254689   
F       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=2,... 3      0.241031   
M       DANN alpha_d=0.75, hidden_size_d=512, num_hidden_d=2... 3      0.197116   
T       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=1,... 3      0.237782   
average DANN                                                           0.232655   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=1.0, hidden_size_d=512, num_hidden_d=2,... 3      0.943021   
F       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=2,... 3      0.935778   
M       DANN alpha_d=0.75, hidden_size_d=512, num_hidden_d=2... 3      0.949503   
T       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=1,... 3      0.935120   
average DANN                                                           0.940855   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=1.0, hidden_size_d=512, num_hidden_d=2,... 3      0.934825   
F       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=2,... 3      0.926454   
M       DANN alpha_d=0.75, hidden_size_d=512, num_hidden_d=2... 3      0.935528   
T       DANN alpha_d=0.1, hidden_size_d=512, num_hidden_d=1,... 3      0.946167   
average DANN                                                           0.935744   

                                                                           test  \
 

### uniform and reweighted DANN

In [20]:
dann = combine_all_csv('../pt/UniRewDANN_default/', arg_names)
model_selection(dann, metric='avg_acc')

train  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.068438   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.045053   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.032988   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.037992   
average DANN                                                           0.046118   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.985007   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.992214   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.994965   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.992595   
average DANN                                                           0.991195   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.982132   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.990858   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.993434   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.993786   
average DANN                                                           0.990053   

                                                                          valid  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.199254   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.224271   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.186698   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.258160   
average DANN                                                           0.217096   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.938975   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.932902   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.947891   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.927010   
average DANN                                                           0.936695   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.929545   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.922737   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.932872   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.939385   
average DANN                                                           0.931135   

                                                                           test  \
 

### reweighted DANN

In [19]:
dann = combine_all_csv('../pt/RewDANN_default/', arg_names)
model_selection(dann, metric='avg_acc')

train  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.100126   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.025330   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.018601   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.029490   
average DANN                                                           0.043387   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.966392   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.994550   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.996375   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.994006   
average DANN                                                           0.987830   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.959944   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.993598   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.995278   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.994976   
average DANN                                                           0.985949   

                                                                          valid  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.189254   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.279763   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.234999   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.306405   
average DANN                                                           0.252605   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.935266   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.930745   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.946817   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.928420   
average DANN                                                           0.935312   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.926606   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.919922   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.931345   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.940206   
average DANN                                                           0.929520   

                                                                           test  \
 

### CDAN

In [16]:
dann = combine_all_csv('../pt/CDAN_default/', arg_names)
model_selection(dann, metric='avg_acc')

train  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.020828   
F       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.022844   
M       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.040774   
T       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.049421   
average CDAN                                                           0.033467   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.995199   
F       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.995089   
M       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.989863   
T       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.986865   
average CDAN                                                           0.991754   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.994286   
F       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.994232   
M       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.986776   
T       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.988993   
average CDAN                                                           0.991072   

                                                                          valid  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.243660   
F       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.280978   
M       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.188789   
T       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.294421   
average CDAN                                                           0.251962   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.938638   
F       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.933381   
M       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.946817   
T       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.929478   
average CDAN                                                           0.937079   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.929621   
F       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.923584   
M       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.932238   
T       CDAN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.941725   
average CDAN                                                           0.931792   

                                                                           test  \
 

### DANN

In [17]:
dann = combine_all_csv('../pt/DANN_default/', arg_names)
model_selection(dann, metric='avg_acc')

train  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 0      0.235291   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.038199   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.090648   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.048719   
average DANN                                                           0.103214   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 0      0.905660   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.990237   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.968247   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.985807   
average DANN                                                           0.962488   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 0      0.886960   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.988528   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.958542   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.988094   
average DANN                                                           0.955531   

                                                                          valid  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 0      0.158477   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.259156   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.165047   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.250716   
average DANN                                                           0.208349   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 0      0.939312   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.934100   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.943594   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.929126   
average DANN                                                           0.936533   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 0      0.930233   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.923675   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.927536   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.941211   
average DANN                                                           0.930664   

                                                                           test  \
 

### Balanced DANN

In [4]:
dann = combine_all_csv('../pt/BlcDANN_default/', arg_names)
model_selection(dann, metric='avg_acc')

train  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.021028   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.090675   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.026365   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.016006   
average DANN                                                           0.038519   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.994605   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.971195   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.993160   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.996719   
average DANN                                                           0.988920   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.995014   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.970022   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.992638   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.997444   
average DANN                                                           0.988780   

                                                                          valid  \
                                                                       avg_loss   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.273840   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.206544   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.213235   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.344284   
average DANN                                                           0.259476   

                                                                                 \
                                                                        avg_acc   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.934929   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.931943   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.946280   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.929126   
average DANN                                                           0.935570   

                                                                                 \
                                                                             f1   
tgt     algo hparams                                            Epoch             
A       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.925741   
F       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 1      0.921023   
M       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 2      0.931271   
T       DANN alpha_d=0.1, hidden_size_d=1024, num_hidden_d=1... 3      0.941314   
average DANN                                                           0.929837   

                                                                           test  \
 

### Balanced ERM (ablation)

In [21]:
blc_erm = combine_all_csv('../pt/BlcERM_search/', arg_names)
model_selection(blc_erm, 'avg_acc')

train                         valid  \
                                       avg_loss   avg_acc        f1  avg_loss   
tgt     algo hparams            Epoch                                           
A       ERM  lr=2e-05, wd=0.001 0      0.172458  0.935742  0.940611  0.171760   
F       ERM  lr=2e-05, wd=0.001 2      0.032502  0.990848  0.990481  0.268561   
M       ERM  lr=2e-05, wd=0.1   2      0.025183  0.993015  0.992482  0.203258   
T       ERM  lr=2e-05, wd=0.01  3      0.011428  0.997604  0.998133  0.348609   
average ERM                            0.060393  0.979302  0.980427  0.248047   

                                                               test            \
                                        avg_acc        f1  avg_loss   avg_acc   
tgt     algo hparams            Epoch                                           
A       ERM  lr=2e-05, wd=0.001 0      0.940661  0.931624  0.328970  0.861375   
F       ERM  lr=2e-05, wd=0.001 2      0.937215  0.926734  0.552936  0.872395   
M       ERM  lr=2e-05, wd=0.1   2      0.948160  0.932588  0.904383  0.790946   
T       ERM  lr=2e-05, wd=0.01  3      0.930183  0.941833  0.683674  0.831966   
average ERM                            0.939055  0.933195  0.617491  0.839170   

                                                 
                                             f1  
tgt     algo hparams            Epoch            
A       ERM  lr=2e-05, wd=0.001 0      0.868055  
F       ERM  lr=2e-05, wd=0.001 2      0.911899  
M       ERM  lr=2e-05, wd=0.1   2      0.841935  
T       ERM  lr=2e-05, wd=0.01  3      0.701315  
average ERM                            0.830801